In [1]:
import pandas as pd
import numpy as np
RANDOM_STATE = 11

There are two datasets, crx.csv and data.csv, please use them to complete the requirements for this assignment below:
1. The labels column in crx and data are "label" and " Diagnosis"

In [2]:
df_crx = pd.read_csv('../data/crx.csv')
df_data = pd.read_csv('../data/data.csv')
df_data = df_data.rename(columns={'Unnamed: 31': '32'})

print('{:<5}'.format('crx'), 'shape: ', df_crx.shape)
print('{:<5}'.format('data'), 'shape: ', df_crx.shape)

crx   shape:  (690, 16)
data  shape:  (690, 16)


In [3]:
df_crx.head(5)

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,att11,att12,att13,att14,att15,label
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43,560,+
2,a,24.5,0.500,u,g,q,h,1.50,t,f,0,f,g,280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120,0,+


In [4]:
df_data.head(5)

,ID,Diagnosis,3,4,5,6,7,8,9,10,...,23,24,25,26,27,28,29,30,31,32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


# data preprocessing

## Deal

In [5]:
def fill_mean(df, col, text):
    mask = (df[col] != text)
    tmp_mean = df.loc[mask, col].astype(float).mean().round(2)
    print('mean: ', tmp_mean)

    mask = (df[col] == text)
    df.loc[mask, col] = tmp_mean
    df[col] = df[col].astype(float)
    return df

df_crx = fill_mean(df_crx, 'att2', '?')
df_crx = fill_mean(df_crx, 'att14', '?')

mean:  31.57
mean:  184.01


In [6]:
def fill_mode(df, col, text):
    mask = (df[col] != text)
    tmp_mode = df.loc[mask, col].mode()[0]
    print('mode: ', tmp_mode)

    mask = (df[col] == text)
    df.loc[mask, col] = tmp_mode
    return df


error_col = ['att1', 'att4', 'att5', 'att6', 'att7']
for col in error_col:
    df_crx = fill_mode(df_crx, col, '?')

mode:  b
mode:  u
mode:  g
mode:  c
mode:  v


## feature

In [7]:
num_types = ['float64', 'int64', 'uint8']
def get_col_types(df):
    ls_num = []
    ls_cat = []
    for i, v in df.dtypes.to_dict().items():
        if v in num_types:
            ls_num.append(i)
        else:
            ls_cat.append(i)
    print('numerical col: ', ls_num)
    print('category col: ', ls_cat)
    return ls_num, ls_cat
ls_num_crx, ls_cat_crx = get_col_types(df_crx)
print('-'*10)
ls_num_data, ls_cat_data = get_col_types(df_data)

numerical col:  ['att2', 'att3', 'att8', 'att11', 'att14', 'att15']
category col:  ['att1', 'att4', 'att5', 'att6', 'att7', 'att9', 'att10', 'att12', 'att13', 'label']
----------
numerical col:  ['ID', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32']
category col:  ['Diagnosis']


### numerical columns

In [8]:
df_crx.loc[:,ls_num_crx].describe()

,att2,att3,att8,att11,att14,att15
count,690.000000,690.000000,690.000000,690.00000,690.000000,690.000000
mean,31.568203,4.758725,2.223406,2.40000,184.014681,1017.385507
std,11.853273,4.978163,3.346513,4.86294,172.159274,5210.102598
min,13.750000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,22.670000,1.000000,0.165000,0.00000,80.000000,0.000000
50%,28.625000,2.750000,1.000000,0.00000,160.000000,5.000000
75%,37.707500,7.207500,2.625000,3.00000,272.000000,395.500000
max,80.250000,28.000000,28.500000,67.00000,2000.000000,100000.000000


In [9]:
df_data.loc[:,ls_num_data].describe()

,ID,3,4,5,6,7,8,9,10,11,...,23,24,25,26,27,28,29,30,31,32
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,3.037183e+07,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,1.250206e+08,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,8.692180e+05,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,9.060240e+05,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,8.813129e+06,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


### categorical columns

In [10]:
df_crx.loc[:,ls_cat_crx].describe()

,att1,att4,att5,att6,att7,att9,att10,att12,att13,label
count,690,690,690,690,690,690,690,690,690,690
unique,2,3,3,14,9,2,2,2,3,2
top,b,u,g,c,v,t,f,f,g,-
freq,480,525,525,146,408,361,395,374,625,383


In [11]:
df_data.loc[:,ls_cat_data].describe()

,Diagnosis
count,569
unique,2
top,B
freq,357


In [12]:
# for i in ls_cat_crx:
#     print(i)
#     print(df_crx[i].value_counts())
#     print('-'*20)

## missing data

In [ ]:
print(df_crx.isna().sum().sum())
print(df_data.isna().sum().sum())

0
0


### seperate X, y

In [13]:
# y mapping
df_crx['label'] = df_crx['label'].replace({"+": 1, "-": -1})
df_data['Diagnosis'] = df_data['Diagnosis'].replace({"M": 1, "B": -1})

In [14]:
y_crx = df_crx.pop('label')
y_data = df_data.pop('Diagnosis')

## normalization

In [15]:
def df_z_score(df):
    for col in df.columns:
        mu = df[col].mean()
        std = df[col].std()
        df[col] = (df[col] - mu) / std
    return df

In [16]:
df_crx[ls_num_crx] = df_z_score(df_crx[ls_num_crx])
df_data[ls_num_data] = df_z_score(df_data[ls_num_data])

/var/folders/q2/mj7nc251111b8s4gb4pc9h880000gn/T/ipykernel_74558/3028910598.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = (df[col] - mu) / std


## one-hot encoding

In [17]:
df_crx = pd.get_dummies(df_crx)
df_crx.shape

(690, 46)

2. Please implement the Linear Classifier from scratch with the update rule in the slide. It means you cannot adopt any existing package like sklearn in this assignment. 

In [18]:
class CLF:
    def __init__(self):
        pass
    @staticmethod
    def check_as_np(*args):
        args = list(args)
        for i in range(len(args)):
            if (type(args[i]) == pd.core.frame.DataFrame) | (type(args[i]) == pd.core.series.Series):
                args[i] = args[i].to_numpy()
        if len(args) == 1:
            return args[0]
        else:
            return args
    @staticmethod
    def union_shuffle(np_a, np_b):
        assert len(np_a) == len(np_b)
        ls = np.random.RandomState(seed=RANDOM_STATE).permutation(len(np_a))
        return np_a[ls], np_b[ls]
        
    def predict(self, X_test):
        X_test = self.check_as_np(X_test)
        y_hat = np.sign(np.dot(X_test, self.weight) + self.intercept).astype(int)
        return y_hat

    def predict_evaluate(self, X_test, y_test):
        y_hat = self.predict(X_test)
        df_cp = pd.DataFrame({
            'y_test': y_test,
            'y_hat': y_hat,
        })
        confusion_matrix = pd.crosstab(y_test, y_hat, rownames=['Actual'], colnames=['Predicted'])
        accuracy = (confusion_matrix.iloc[0, 0] + confusion_matrix.iloc[1, 1]) / confusion_matrix.sum().sum()
        precision = confusion_matrix.iloc[1, 1] / confusion_matrix.iloc[:, 1].sum()
        recall = confusion_matrix.iloc[1, 1] / confusion_matrix.iloc[1, :].sum()
        print("{:<10}:".format("Accuracy"), accuracy)
        print("{:<10}:".format("Precision"), precision)
        print("{:<10}:".format("Recall"), recall)
        return confusion_matrix, accuracy, precision, recall

In [19]:
class LinearCLF(CLF):
    def __init__(self, learning_rate=1, max_iter=50, constrait=0):
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.constrait = constrait

    def fit(self, X, y):
        X, y = self.check_as_np(X, y)
        self.X = X
        self.y = y
        self.weight = np.zeros(X.shape[1])
        self.intercept = 0
        
        for i in range(self.max_iter):
            self.fit_once()

    def fit_once(self):
        # shuffle
        X_fit, y_fit = self.X, self.y
        X_fit, y_fit = self.union_shuffle(self.X, self.y)
        for index, (X, y) in enumerate(zip(X_fit, y_fit)):
            step = 0
            while ((np.dot(self.weight, X)+ self.intercept) * y) <= self.constrait:
                # not correct
                self.weight = self.weight + self.learning_rate * (y * X)
                self.intercept = self.intercept + y
                step += 1


In [20]:
# df_crx
linear_clf = LinearCLF()
linear_clf.fit(df_crx, y_crx)
# y_hat = linear_clf.predict(df_crx)
linear_clf.predict_evaluate(df_crx, y_crx)

Accuracy  : 0.8260869565217391
Precision : 0.7791044776119403
Recall    : 0.8501628664495114


(Predicted   -1    1
 Actual             
 -1         309   74
  1          46  261,
 0.8260869565217391,
 0.7791044776119403,
 0.8501628664495114)

In [21]:
# df_data
linear_clf = LinearCLF()
linear_clf.fit(df_data, y_data)
linear_clf.predict_evaluate(df_data, y_data)

Accuracy  : 0.9806678383128296
Precision : 0.9674418604651163
Recall    : 0.9811320754716981


(Predicted   -1    1
 Actual             
 -1         350    7
  1           4  208,
 0.9806678383128296,
 0.9674418604651163,
 0.9811320754716981)

3. When the "J=WX+b" could be represented as the matrix form for the linear classifier, please find the solution by solving this equation using least-squared manner. Also, please implement it and make a comparison between this method and the previous one implemented in 2.


In [22]:
class LeastSquaredCLF(CLF):
    @staticmethod
    def expand_b0(X, value):
        insert_index = X.shape[1]
        return np.insert(X, insert_index, value, axis=1)
    def __init__(self):
        pass
    def fit(self, X, y):
        self.X, self.y = self.check_as_np(X, y)
        self.X = self.expand_b0(self.X, 1)
        self.weight = np.linalg.pinv(np.dot(self.X.T, self.X)).dot(self.X.T).dot(self.y)
        self.intercept = self.weight[-1]
        self.weight = np.delete(self.weight, -1)
        return self.weight, self.intercept

    # def predict_evaluate(self, X_test, y_test):
    #     X_test, y_test = self.check_as_np(X_test, y_test)
    #     X_test = self.expand_b0(X_test, 1)
    #     return super().predict_evaluate(X_test, y_test)

In [23]:
# df_crx
least_squared_clf = LeastSquaredCLF()
least_squared_clf.fit(df_crx, y_crx)
least_squared_clf.predict_evaluate(df_crx, y_crx)

Accuracy  : 0.863768115942029
Precision : 0.7917808219178082
Recall    : 0.9413680781758957


(Predicted   -1    1
 Actual             
 -1         307   76
  1          18  289,
 0.863768115942029,
 0.7917808219178082,
 0.9413680781758957)

In [24]:
# df_data
least_squared_clf = LeastSquaredCLF()
least_squared_clf.fit(df_data, y_data)
least_squared_clf.predict_evaluate(df_data, y_data)

Accuracy  : 0.9648506151142355
Precision : 0.9897959183673469
Recall    : 0.9150943396226415


(Predicted   -1    1
 Actual             
 -1         355    2
  1          18  194,
 0.9648506151142355,
 0.9897959183673469,
 0.9150943396226415)

4. Please implement the voted perception and make the performance comparison between methods.

In [25]:
class VotedPerceptronCLF(LinearCLF):
    def __init__(self, lr=0.1, max_iter=50, method='vote'):
        super().__init__(lr, max_iter)
        self.store_weights = np.array([])
        self.store_intercepts = np.array([])
        self.store_votes = np.array([])
        self.method = method
    def fit_once(self):
        # shuffle
        X_fit, y_fit = self.union_shuffle(self.X, self.y)
        for index, (X, y) in enumerate(zip(X_fit, y_fit)):
            step = 0
            while ((np.dot(self.weight, X)+ self.intercept) * y) <= 0:
                # not correct
                self.weight = self.weight + self.learning_rate * (y * X)
                self.intercept = self.intercept + y
                step += 1
            # print(f"index: {index}, store weight")
            self.store_model(X_fit, y_fit)
    def store_model(self, X_fit, y_fit):
        y_hat = super().predict(X_fit)
        vote = np.equal(y_fit, y_hat).sum()
        self.store_votes = np.append(self.store_votes, np.array(vote))
        self.store_intercepts = np.append(self.store_intercepts, np.array(self.intercept))
        if len(self.store_weights) == 0:
            self.store_weights = np.expand_dims(self.weight, axis=0)
        else:
            self.store_weights = np.append(self.store_weights, np.expand_dims(self.weight, axis=0), axis=0)
    def predict(self, X_test):
        X_test = self.check_as_np(X_test)
        if self.method=='vote':
            y_hat = np.sign(np.dot(X_test, self.store_weights.T) + self.store_intercepts).astype(int)
            y_hat = np.sign((y_hat * self.store_votes).sum(axis=1))
        elif self.method=='average':
            avg_weight = np.dot(self.store_weights.T, self.store_votes) / self.store_votes.sum()
            
            avg_intercept = (self.store_intercepts * self.store_votes).sum() / (self.store_votes.sum())
            y_hat = np.sign(np.dot(X_test, avg_weight) + avg_intercept)
        return y_hat

In [26]:
# df_crx
print('voted')
voted_perceptron_clf = VotedPerceptronCLF(1, 50)
voted_perceptron_clf.fit(df_crx, y_crx)
voted_perceptron_clf.predict_evaluate(df_crx, y_crx)

voted
Accuracy  : 0.8782608695652174
Precision : 0.8562300319488818
Recall    : 0.8729641693811075


(Predicted  -1.0   1.0
 Actual               
 -1          338    45
  1           39   268,
 0.8782608695652174,
 0.8562300319488818,
 0.8729641693811075)

In [27]:
print('avg')
avg_perceptron_clf = VotedPerceptronCLF(1, 50, 'average')
avg_perceptron_clf.fit(df_crx, y_crx)
avg_perceptron_clf.predict_evaluate(df_crx, y_crx)

avg
Accuracy  : 0.8782608695652174
Precision : 0.8562300319488818
Recall    : 0.8729641693811075


(Predicted  -1.0   1.0
 Actual               
 -1          338    45
  1           39   268,
 0.8782608695652174,
 0.8562300319488818,
 0.8729641693811075)

In [28]:
# df_data
print('vote')
voted_perceptron_clf = VotedPerceptronCLF(1, 50)
voted_perceptron_clf.fit(df_data, y_data)
voted_perceptron_clf.predict_evaluate(df_data, y_data)

vote
Accuracy  : 0.984182776801406
Precision : 0.9856459330143541
Recall    : 0.9716981132075472


(Predicted  -1.0   1.0
 Actual               
 -1          354     3
  1            6   206,
 0.984182776801406,
 0.9856459330143541,
 0.9716981132075472)

In [29]:
print('avg')
avg_perceptron_clf = VotedPerceptronCLF(1, 50, 'average')
avg_perceptron_clf.fit(df_data, y_data)
avg_perceptron_clf.predict_evaluate(df_data, y_data)

avg
Accuracy  : 0.9824253075571178
Precision : 0.9809523809523809
Recall    : 0.9716981132075472


(Predicted  -1.0   1.0
 Actual               
 -1          353     4
  1            6   206,
 0.9824253075571178,
 0.9809523809523809,
 0.9716981132075472)

5. With minimizing the ||w||^2, it should drive the marginal to be maximized as well. Please implement the linear classifier with the minimum ||w||^2 property and verify whether the margin of this version is larger than that of the conventional linear classifier or not. 

In [46]:
class preSVM(LinearCLF):
    def __init__(self, lr=0.1, max_iter=50, constrait=0):
        super().__init__(lr, max_iter)

    def fit(self, X, y):
        X, y = self.check_as_np(X, y)
        self.X = X
        self.y = y
        self.intercept = 0
        self.weight = np.zeros(X.shape[1])
        self.store_weights = np.empty((0, X.shape[1]))
        self.store_intercepts = np.array([])
        self.store_distances = np.array([])
        
        for i in range(self.max_iter):
            self.fit_once()
            _, acc, _, _ = self.predict_evaluate(X, y)
            if acc > 0.8:
                self.store_weights = np.append(
                    self.store_weights,
                    np.expand_dims(self.weight, axis=0), axis=0)
                self.store_intercepts = np.append(
                    self.store_intercepts,
                    np.array(self.intercept))
    
    def calc_distance(self):
        for weight, intercept in zip(self.store_weights, self.store_intercepts):
            distance = 0
            for index, (X, y) in enumerate(zip(self.X, self.y)):
                y_hat = (np.dot(X, weight) + intercept)
                if (y * y_hat) > 0:
                    distance += (y * y_hat)
            self.store_distances = np.append(self.store_distances, np.array(distance))
    def get_w_distance_df(self):
        weight_power_sum = np.power(self.store_weights, 2).sum(axis=1)

        return pd.DataFrame({
            'weight_power_sum': weight_power_sum, 
            'distance': self.store_distances})

# test
pre_svm = preSVM()
pre_svm.fit(df_crx, y_crx)
pre_svm.calc_distance()
df_w_distance = pre_svm.get_w_distance_df()
df_w_distance

Accuracy  : 0.8173913043478261
Precision : 0.8549019607843137
Recall    : 0.7100977198697068
Accuracy  : 0.7753623188405797
Precision : 0.8653846153846154
Recall    : 0.5863192182410424
Accuracy  : 0.7623188405797101
Precision : 0.668235294117647
Recall    : 0.9250814332247557
Accuracy  : 0.8115942028985508
Precision : 0.7451523545706371
Recall    : 0.8762214983713354
Accuracy  : 0.8115942028985508
Precision : 0.7610619469026548
Recall    : 0.8403908794788274
Accuracy  : 0.7956521739130434
Precision : 0.75
Recall    : 0.8110749185667753
Accuracy  : 0.8420289855072464
Precision : 0.8193548387096774
Recall    : 0.8273615635179153
Accuracy  : 0.7115942028985507
Precision : 0.9576271186440678
Recall    : 0.36807817589576547
Accuracy  : 0.7260869565217392
Precision : 0.9609375
Recall    : 0.4006514657980456
Accuracy  : 0.8246376811594203
Precision : 0.801948051948052
Recall    : 0.8045602605863192
Accuracy  : 0.827536231884058
Precision : 0.8071895424836601
Recall    : 0.8045602605863192
Ac

,weight_power_sum,distance
0,7.591072,1052.217768
1,11.040869,935.121443
2,11.977887,952.465682
3,14.877480,1023.638305
4,18.769137,1028.144081
5,18.934526,1056.218082
6,20.464304,1128.189020
7,20.494831,937.388086
8,20.989903,1031.126989
9,21.599441,1031.129646


In [47]:
df_w_distance.corr()

,weight_power_sum,distance
weight_power_sum,1.000000,0.570603
distance,0.570603,1.000000


6. Based on 5, please add the slack variable term in the linear classifier and find the most effective weighting value C.

7. Use any existing SVM package (libSVM, sklearn, or Matlab SVM), please make a performance comparison between the built-in SVM and your implementation.

All the source code should be uploaded to Github with public access. The performance comparison should be organized into a formal report with the standard template indicated in Moodle. 

In [37]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
clf = SVC(gamma='auto')
clf.fit(df_crx, y_crx)
y_hat = clf.predict(df_crx)
cm = confusion_matrix(y_crx, y_hat)
acc = (cm[1, 1] + cm[0, 0]) / cm.sum()
precision = cm[1, 1] / cm[:, 1].sum()
recall = cm[1, 1] / cm[1, :].sum()
print("{:>10}:".format("Accuracy"), acc)
print("{:>10}:".format("Precision"), precision)
print("{:>10}:".format("Recall"), recall)
print(cm)

  Accuracy: 0.8565217391304348
 Precision: 0.787292817679558
    Recall: 0.9283387622149837
[[306  77]
 [ 22 285]]
